<center>
    <img src="https://raw.githubusercontent.com/dvgodoy/PyTorch101_ODSC_Europe2020/master/images/linear_dogs.jpg" width="800"> 
</center>  

# Yet another math for DS course: домашнее задание 6

**ФИО:** Теплов Вячеслав Сергеевич

__Жёсткий дедлайн:__ 23:59MSK 24.11.2024

## Формат сдачи

Сам ноутбук называйте в формате hw-07-USERNAME.ipynb, где USERNAME — ваши фамилия и имя. Файл надо будет сдать в anytask.

## Задача №1: точность вычислений

С точки зрения математики матричные разложения являются точными: произведение сомножителей всегда равняется исходной матрицы $A$. К сожалению, на практике этом часто мешает вычислительная погрешность. 

Для $LU$ разложения $l_2$-норма ошибки ошибки $||\delta A|| = ||A - LU||$ удовлетворяет следующей оценке:

$$||\delta A|| \leqslant ||L|| \cdot ||U|| \cdot O(\varepsilon_{machine})$$

А нормы $L$ и $U$ могут быть совсем нехорошими.

__[2 балла] Задание 1.1__ Рассмотрим следующее LU-разложение:

$$
\begin{pmatrix}
1 & 0\\
10^{20} & 1
\end{pmatrix}
\cdot
\begin{pmatrix}
10^{-20} & 1\\
0 & 1 - 10^{20}
\end{pmatrix}
$$
и
$$
\begin{pmatrix}
1. & 0.\\
10.^{20} & 1.
\end{pmatrix}
\cdot
\begin{pmatrix}
10.^{-20} & 1.\\
0. & 1. - 10.^{20}
\end{pmatrix}
$$

Во втором случае мы специально указали Питону (поставив везде десятичные точки), что хотим работать не с целыми числами, а с числами с плавающей точкой. Перемножьте полученные матрицы $L$ и $U$. Как ответ вам кажется правильным, почему?

> __Ответ:__ Матрица с целочисленными значениями более правильная, так как отсутствует погрешность при вычислении чисел с плавающей точкой. При вычислении произведения матриц с плавающими точками могут возникнуть ошибки при округлении, например, если приведем результирующую матрицу к типу int.

In [1]:
import numpy as np

L1 = np.array([[1, 0], [10 ** 20, 1]])
U1 = np.array([[10 ** -20, 1], [0, 1 - 10 ** 20]])

L2 = np.array([[1., 0.], [10. ** 20, 1.]])
U2 = np.array([[10. ** -20, 1.], [0., 1. - 10. ** 20]])

A1 = L1 @ U1
A2 = L2 @ U2

print("Результат для целых чисел (A1):")
print(A1)

print("\nРезультат для чисел с плавающей точкой (A2):")
print(A2)

Результат для целых чисел (A1):
[[1e-20 1]
 [1.0 1]]

Результат для чисел с плавающей точкой (A2):
[[1.e-20 1.e+00]
 [1.e+00 0.e+00]]


Отметим, что в реальных вычислениях матричные элементы почти наверняка с самого начала будут числами с плавающей точкой (а не целыми).

**Выход: LU-разложение с выбором главного элемента (по столбцу)**

Каждый раз ищем максимум в столбце и переставляем соответствующую строку наверх.

$$\begin{pmatrix}
b_{11} & \dots & b_{1i} & b_{1,i+1} & \dots & b_{1n}\\
 & \ddots & \vdots & \vdots & & \vdots\\
 & & \color{blue}{b_{ii}} & \color{blue}{b_{i,i+1}} & \dots & \color{blue}{b_{in}} \\
 & & b_{i+1,i} & b_{i+1,i+1} & \dots & b_{i+1,n}\\
 & & \vdots & \vdots &  & \vdots \\
 & & \color{green}{b_{ji}} & \color{green}{b_{j,i+1}} & \dots & \color{green}{b_{jn}} \\
 & & \vdots & \vdots & & \vdots\\
\end{pmatrix}\longrightarrow
\begin{pmatrix}
b_{11} & \dots & b_{1i} & b_{1,i+1} & \dots & b_{1n}\\
 & \ddots & \vdots & \vdots & & \vdots\\
 & & \color{green}{b_{ji}} & \color{green}{b_{j,i+1}} & \dots & \color{green}{b_{jn}} \\
 & & b_{i+1,i} & b_{i+1,i+1} & \dots & b_{i+1,n}\\
 & & \vdots & \vdots &  & \vdots \\
 & & \color{blue}{b_{ii}} & \color{blue}{b_{i,i+1}} & \dots & \color{blue}{b_{in}} \\
 & & \vdots & \vdots & & \vdots\\
\end{pmatrix}\longrightarrow$$
$$\longrightarrow\begin{pmatrix}
b_{11} & \dots & b_{1i} & b_{1,i+1} & \dots & b_{1n}\\
 & \ddots & \vdots & \vdots & & \vdots\\
 & & \color{green}{b_{ji}} & \color{green}{b_{j,i+1}} & \dots & \color{green}{b_{jn}} \\
 & & 0 & b'_{i+1,i+1} & \dots & b'_{i+1,n}\\
 & & \vdots & \vdots &  & \vdots \\
 & & 0 & b'_{i,i+1} & \dots & b'_{in} \\
 & & \vdots & \vdots & & \vdots
\end{pmatrix}$$

Надо сказать, что примерно так вы все и решали системы на первом курсе университета! Именно наибольший, а не первый ненулевой элемент столбца берётся потому, что чем больше число - тем меньшие погрешности потенциально вносит деление на него.

Что при этом происходит? Перестановка строк матрицы равносильна умножению её слева на матрицу соответствующей перестановки. Таким образом, мы получаем равенство

$$L_nP_nL_{n-1}P_{n-1}\ldots L_2P_2L_1P_1 A = U\qquad\qquad(1)$$

где $L_1,\ldots,L_n$ - некоторые нижнетреугольные матрицы.

**Вопрос:** Ну, и где здесь матрица $L$?!

**Ответ:** Введём новые матрицы

\begin{align*}
L'_n &= L_n\\
L'_{n-1} &= P_nL_nP_{n-1}\\
L'_{n-2} &= P_nP_{n-1}L_{n-1}P_n^{-1}P_{n-1}^{-1}\\
&\ldots\\
L'_1 &= P_nP_{n-1}\ldots P_2L_1P_2^{-1}\ldots P_{n-1}^{-1}P_n^{-1}
\end{align*}

**Упражнение.** Матрицы $L'_i$ тоже нижнетреугольные!

Тогда левая часть (1) перепишется в виде

$$\underbrace{L'_nL'_{n-1}\ldots L'_1}_{:=L^{-1}}\underbrace{P_nP_{n-1}\ldots P_1}_{:=P^{-1}}\cdot A$$

**Итог:** разложение вида
$$A = PLU$$
где $P$ - матрица перестановки.

Функция `scipy.linalg.lu` в Питоне находит именно такое разложение!

Все элементы $L$ не превосходят $1$, так что $||L||]\leqslant 1$. При этом
$$||\Delta A|| \leqslant ||A||\cdot O(\rho \varepsilon_{machine}),$$
где
$$\rho = \frac{\max_{i,j}|u_{ij}|}{\max_{i,j}|a_{ij}|}$$
Это число называется *фактором роста матрицы*.

Но что, если это отношение велико?

__[2 балла] Задание 1.2__  Сгенерируйте матрицу $500\times500$, имеющую вид

$$\begin{pmatrix}
1 & 0 & 0 & \cdots & 0 & 0 & 1\\
-1 & 1 & 0 &  &  & 0 & 1\\
-1 & -1 & 1 & 0  &  & 0 & 1\\
\vdots & & \ddots & \ddots  & \ddots & \vdots & \vdots \\
-1 & -1 & -1 & \ddots & 1 & 0 & 1\\
-1 & -1 & -1 &  & -1 & 1 & 1\\
-1 & -1 & -1 & \cdots & -1 & -1 & 1
\end{pmatrix}$$

Например, вы можете сгенерировать сначала нулевую матрицу нужного размера, а потом заполнить её клетки правильными числами.

Найдите её PLU-разложение и QR-разложение. Убедитесь, что $P = E$. Вычислите $||A - LU||_2$ и $||A - QR||_2$. Чему равен фактор роста матрицы $A$?

In [2]:
import numpy as np
from scipy.linalg import lu, qr, norm

n = 500
A = np.zeros((n, n))

for i in range(n):
    A[i, i] = 1
    A[i, -1] = 1
    if i > 0:
        A[i, :i] = -1

# PLU-разложение
P, L, U = lu(A)

P, L, U

(array([[1., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 0., 1.]]),
 array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
        [-1.,  1.,  0., ...,  0.,  0.,  0.],
        [-1., -1.,  1., ...,  0.,  0.,  0.],
        ...,
        [-1., -1., -1., ...,  1.,  0.,  0.],
        [-1., -1., -1., ..., -1.,  1.,  0.],
        [-1., -1., -1., ..., -1., -1.,  1.]]),
 array([[1.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
         0.00000000e+000, 0.00000000e+000, 1.00000000e+000],
        [0.00000000e+000, 1.00000000e+000, 0.00000000e+000, ...,
         0.00000000e+000, 0.00000000e+000, 2.00000000e+000],
        [0.00000000e+000, 0.00000000e+000, 1.00000000e+000, ...,
         0.00000000e+000, 0.00000000e+000, 4.00000000e+000],
        ...,
        [0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
         1

In [3]:
# QR-разложение
Q, R = qr(A)
Q, R

(array([[-4.47213595e-02,  4.45332635e-01,  1.94543248e-01, ...,
          8.17790626e-17,  1.17044185e-16,  8.66025404e-01],
        [ 4.47213595e-02, -8.93353395e-01,  9.72716239e-02, ...,
          3.95796397e-17,  5.97203116e-17,  4.33012702e-01],
        [ 4.47213595e-02,  2.68812456e-03, -9.74873473e-01, ...,
          1.96576291e-17,  2.98072803e-17,  2.16506351e-01],
        ...,
        [ 4.47213595e-02,  2.68812456e-03,  2.15723360e-03, ...,
         -8.16496581e-01, -2.68173596e-18,  1.38777878e-17],
        [ 4.47213595e-02,  2.68812456e-03,  2.15723360e-03, ...,
          4.08248290e-01, -7.07106781e-01, -1.94289029e-16],
        [ 4.47213595e-02,  2.68812456e-03,  2.15723360e-03, ...,
          4.08248290e-01,  7.07106781e-01, -1.38777878e-17]]),
 array([[-2.23606798e+01, -2.22265157e+01, -2.21817943e+01, ...,
         -4.47213595e-02,  0.00000000e+00,  2.22712371e+01],
        [ 0.00000000e+00, -2.23203943e+00, -1.33330978e+00, ...,
         -2.68812456e-03,  0.00000000e

In [4]:
# P == E
np.allclose(P, np.eye(n))

True

In [5]:
error_LU = norm(A - L @ U, 2)
error_QR = norm(A - Q @ R, 2)

print("L2 норма ||A - LU||:", error_LU)
print("L2 норма ||A - QR||:", error_QR)

L2 норма ||A - LU||: 5.0768322940305e+114
L2 норма ||A - QR||: 6.534508843060089e-13


In [6]:
growth_factor = norm(U, np.inf) / norm(A, np.inf) # берем максимальную норму для max(|u|) / max(|a|) 
print("Фактор роста матрицы A:", growth_factor)

Фактор роста матрицы A: 3.273390607896142e+147


К счастью, на практике так редко очень бывает (чёрт его знает почему). Тем не менее, QR-разложение всё-таки лучше. Теоретическая оценка для ошибки QR-разложения имеет вид

$$||A - QR||_2 \leqslant ||A||_2\cdot O(\varepsilon_{machine})$$

__[2 балла] Задание 1.3__  Рассмотрим *матрицу Паскаля* $S_n = \left(C_{i + j}^i\right)$ ($i,j = 0,\ldots,n-1$).

Каково её LU-разложение? Выведите формулы для матриц L и U и приведите краткое обоснование прямо в ноутбуке. Не пользуйтесь функцией `scipy.linalg.lu`, чтобы его "угадать": матрица P будет отлична от единичной, и вы получите не то, что хотели.

Каков её определитель?

> __Ответ:__ Ваши формулы (ручное решение задачки вбитое в маркдаун)

1. **Матрица $L$** — это нижнетреугольная матрица:
   $$
   L[i, j] = 
   \begin{cases}
   C_i^j, & \text{если } i \ge j, \\
   0, & \text{если } i < j
   \end{cases}
    $$
   где $C_i^j$ — биномиальный коэффициент, определяющий количество способов выбрать $j$ элементов из $i$.

2. **Матрица $U$** — это верхнетреугольная матрица:
   $$
   U[i, j] = 
   \begin{cases}
   1, & \text{если } i \le j, \\
   0, & \text{если } i > j
   \end{cases}
   $$

3. Поскольку $U$ является верхнетреугольной матрицей с единицами на диагонали, её определитель равен 1. Таким образом, определитель $S_n$ равен произведению диагональных элементов $L$, что даёт:
$$
\det(S_n) = \prod_{i=0}^{n-1} L[i, i] = 1
$$

__[4 балла] Задание 1.4__ Напишите функцию `my_pascal(n)`, генерирующую матрицу Паскаля размера $n\times n$.

In [7]:
from math import comb


def my_pascal(n):
    pascal_matrix = np.zeros((n, n), dtype=int)

    for i in range(n):
        for j in range(n):
            pascal_matrix[i, j] = int(comb(i + j, i))

    return pascal_matrix

Найдите норму разности $||A - PLU||_2$. Не такая уж и большая, правда?

In [8]:
A = my_pascal(10)
P, L, U = lu(A)
PLU = P @ L @ U
norm_A_PLU = norm(A - PLU, 2)

print("L2 норма ||A - PLU||:", norm_A_PLU)

L2 норма ||A - PLU||: 6.164262559385983e-12


Теперь попросим компьютер вычислить определитель матрицы Паскаля $30\times30$ и решить простенькую систему уравнений:

In [9]:
from numpy.linalg import det, solve

n = 30
A = my_pascal(n)
determinant = det(A)

print("Определитель матрицы Паскаля 30x30: ", determinant)

Определитель матрицы Паскаля 30x30:  -3.400444993542791e+51


In [12]:
determinant_Q_R = det(Q @ R)
determinant_QR = det(Q) * det(R)

print("Определитель матрицы Q @ R: ", determinant_Q_R)
print("Определитель Q * определитель R: ", determinant_QR)

Определитель матрицы Q @ R:  -6.0967779539099024e+50
Определитель Q * определитель R:  1.321655894306892e+46


In [10]:
# Попробуем решить A * x = b
x = np.ones(n)
b = A @ x
x_computed = solve(A, b)

print("L2 норма для A * x = b:", norm(x_computed - x, 2))

L2 норма для A * x = b: 55374888.97332912


Так себе ошибка. Теперь попробуем сделать это с помощью QR-разложения. Станет ли лучше?

In [11]:
Q, R = qr(A)
# Решаем R * x = Q.T * b
x_computed = solve(R, Q.T @ b)

print("L2 норма для R * x = Q.T * b:", norm(x_computed - x, 2))

L2 норма для R * x = Q.T * b: 123485825.24152747


Объясните полученные неприятные результаты.

> __Ответ:__ Не знаю, какие тут выводы можно сделать. Ошибка при вычислениях с помощью QR-разложения больше вероятно из-за того, что приходится делать дополнительные шаги, из-за этого точность ещё больше падает.